In [13]:
# connect to Weaviate
import weaviate

weaviate_client = weaviate.connect_to_local()

In [17]:
code_collection = weaviate_client.collections.get("Code")
docs_collection = weaviate_client.collections.get("Docs")

In [18]:
# define hybrid search function

def code_search(query: str) -> str:
    """Sends a query to Weaviate's Hybrid Search. Parases the response into a {k}:{v} string."""
    
    response = code_collection.query.hybrid(query, limit=5)
    
    stringified_response = ""
    for idx, o in enumerate(response.objects):
        stringified_response += f"Search Result: {idx+1}:\n"
        for prop in o.properties:
            stringified_response += f"{prop}:{o.properties[prop]}"
        stringified_response += "\n"
    
    return stringified_response

def docs_search(query: str) -> str:
    """Sends a query to Weaviate's Hybrid Search. Parases the response into a {k}:{v} string."""
    
    response = docs_collection.query.hybrid(query, limit=5)
    
    stringified_response = ""
    for idx, o in enumerate(response.objects):
        stringified_response += f"Search Result: {idx+1}:\n"
        for prop in o.properties:
            stringified_response += f"{prop}:{o.properties[prop]}"
        stringified_response += "\n"
    
    return stringified_response

In [35]:
import ollama
from typing import List, Dict

tools_schema=[
    {
        'type': 'function',
        'function': {
            'name': 'code_search',
            'description': "Imagine having a super-powered magnifying glass that lets you instantly pinpoint specific lines of code within a vast software project, like finding a needle in a digital haystack. code_search does just that. It enables you to search directly within the codebase using keywords, phrases, or even code snippets themselves, making it an invaluable tool for developers, support teams, and anyone needing to quickly navigate and understand the inner workings of the software.",
            'parameters': {
              'type': 'object',
              'properties': {
                'query': {
                  'type': 'string',
                  'description': 'The search query.',
                },
              },
              'required': ['query'],
            },
        },
    },
    {
        'type': 'function',
        'function': {
            'name': 'docs_search',
            'description': "Think of it as having a knowledgeable software expert available at your fingertips, ready to answer any questions you have about how to use the software. documentation_search makes that possible. It lets you search through the human-readable documentation associated with the code, providing clear explanations, instructions, and examples to help you learn and use the software effectively. Whether you're a beginner or a seasoned user, documentation_search is your go-to resource for getting the most out of the software.",
            'parameters': {
              'type': 'object',
              'properties': {
                'query': {
                  'type': 'string',
                  'description': 'The search query.',
                },
              },
              'required': ['query'],
            },
        },
    }
]

tool_mapping = {
    "code_search": code_search,
    "docs_search": docs_search
}

def ollama_generation_with_tools(user_message: str,
                                 tools_schema: List, tool_mapping: Dict,
                                 model_name: str = "llama3.1") -> str:
    messages=[{
        "role": "user",
        "content": user_message
    }]
    response = ollama.chat(
        model=model_name,
        messages=messages,
        tools=tools_schema
    )
    if not response["message"].get("tool_calls"):
        return response["message"]["content"]
    else:
        for tool in response["message"]["tool_calls"]:
            selected_function = tool["function"]["name"]
            function_to_call = tool_mapping[selected_function]
            print(f"Calling function \033[92m{selected_function}\033[0m...")
            function_response = function_to_call(tool["function"]["arguments"]["query"])
            messages.append({
                "role": "tool",
                "content": function_response,
            })
    
    final_response = ollama.chat(model=model_name, messages=messages)
    return final_response["message"]["content"]

In [36]:
answer = ollama_generation_with_tools("Can you create a RAG program with DSPy and Weaviate?",
                            tools_schema=tools_schema, tool_mapping=tool_mapping)

print(f"\n{answer}")

Calling function docs_search...

Based on the provided code snippets, I will create a RAG program using DSPy and Weaviate.

```python
import dspy
from weaviate import Client
from dspy.teleprompt import BootstrapFewShot
from dspy.evaluate.evaluate import Evaluate

# Initialize Weaviate client
client = Client("http://localhost:8080")

# Define validation logic
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

# Set up a basic teleprompter
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)

# Define retrieval module
class Retrieve(dspy.Module):
    def __init__(self, k=3):
        super().__init__()
        self.k = k
    
    def forward(self, question):
        query = client.query(
            "Get",
            vector={
                "vector": {
                    "value": question,
         

In [38]:
answer = ollama_generation_with_tools("What is the Avatar optimizer?",
                            tools_schema=tools_schema, tool_mapping=tool_mapping)

print(f"\n{answer}")

Calling function code_search...

Based on the search results, it appears that the Avatar optimizer is a class or module in a Python package that is used to optimize some process or algorithm. The exact details are not provided in the search results, but here are some key points that can be inferred:

1. **Avatar class**: There is a class named `Avatar` that takes several parameters in its constructor, including `signature`, `tools`, `max_iters`, and `verbose`. This suggests that the Avatar optimizer has various parameters that can be adjusted to control its behavior.
2. **Tool management**: The Avatar class seems to have some tool management functionality, as it creates a "Finish" tool and appends it to a list of tools.
3. **Actor creation**: The Avatar class creates an actor (presumably an instance of some predictive model) based on the input fields and signature. This actor is then cloned for later use.
4. **Verbose mode**: The Avatar optimizer can be run in verbose mode, which sugge